# 事前準備

ライブラリのインストール

In [ ]:
!pip install numexpr

# 港湾統計マスタの前処理

港湾統計2021のcsvを読み込む

In [ ]:
import pandas as pd
df_matster = pd.read_csv('kouwan2021.csv', header=0, encoding="shift-jis",dtype=str)
df_matster

必要な列だけ抽出

In [87]:
# 抽出する列のリスト指定
columns = ['調査港','施設','公専','用途','船舶隻数','船舶総ﾄﾝ数','係留時間','出荷品名','出荷ﾄﾝ数','入荷品名','入荷ﾄﾝ数']

df_matster =  df_matster[columns]
df_matster

,調査港,施設,公専,用途,船舶隻数,船舶総ﾄﾝ数,係留時間,出荷品名,出荷ﾄﾝ数,入荷品名,入荷ﾄﾝ数
1,202,1241,1,T84,0,0,3,NaN,0,161,1600
4,202,1212,1,t85,1,749,4,NaN,0,281,1502
5,202,1221,1,T71,1,9910,64,NaN,0,121,10700
6,202,1231,1,T71,1,600,3,NaN,0,222,398
8,202,1215,1,T85,1,748,3,NaN,0,281,1002
...,...,...,...,...,...,...,...,...,...,...,...
44773,202,1249,1,T69,0,0,2,NaN,0,NaN,0
44774,202,1249,1,T69,0,0,0,NaN,0,NaN,0
44787,204,1256,1,T71,1,749,3,222,381,NaN,0
44788,204,1254,1,T71,0,0,4,NaN,0,481,666


調査港、施設（岸壁）

In [88]:
# 抽出する港区コードのリスト指定
port_id = ['201','202','203','204','205','402','403','404','405','406']

df_matster = df_matster[df_matster['調査港'].isin(port_id)]
df_matster

,調査港,施設,公専,用途,船舶隻数,船舶総ﾄﾝ数,係留時間,出荷品名,出荷ﾄﾝ数,入荷品名,入荷ﾄﾝ数
1,202,1241,1,T84,0,0,3,NaN,0,161,1600
4,202,1212,1,t85,1,749,4,NaN,0,281,1502
5,202,1221,1,T71,1,9910,64,NaN,0,121,10700
6,202,1231,1,T71,1,600,3,NaN,0,222,398
8,202,1215,1,T85,1,748,3,NaN,0,281,1002
...,...,...,...,...,...,...,...,...,...,...,...
44773,202,1249,1,T69,0,0,2,NaN,0,NaN,0
44774,202,1249,1,T69,0,0,0,NaN,0,NaN,0
44787,204,1256,1,T71,1,749,3,222,381,NaN,0
44788,204,1254,1,T71,0,0,4,NaN,0,481,666


対象施設コード

In [89]:
# 抽出する港区コードのリスト指定
shisetsu_id = ['1285','1270','1280','1275','1279','1203','1207','1222','1213','1214','1215','1219','1208','1231', \
               '1232','1241','1242','1223','1212','1220','1221','1249','1281','1250','1289','1269','1264','1263', \
               '1262','1258','1259','1252','1251','1254','1255','1256','1153','1152','1154','1197','1178','1179', \
               '1159','1160','1170','1171','1199','1181','1182','1195','1155','1156','1157','1158','1198']

df_matster = df_matster[df_matster['施設'].isin(shisetsu_id)]
df_matster

,調査港,施設,公専,用途,船舶隻数,船舶総ﾄﾝ数,係留時間,出荷品名,出荷ﾄﾝ数,入荷品名,入荷ﾄﾝ数
1,202,1241,1,T84,0,0,3,NaN,0,161,1600
4,202,1212,1,t85,1,749,4,NaN,0,281,1502
5,202,1221,1,T71,1,9910,64,NaN,0,121,10700
6,202,1231,1,T71,1,600,3,NaN,0,222,398
8,202,1215,1,T85,1,748,3,NaN,0,281,1002
...,...,...,...,...,...,...,...,...,...,...,...
44773,202,1249,1,T69,0,0,2,NaN,0,NaN,0
44774,202,1249,1,T69,0,0,0,NaN,0,NaN,0
44787,204,1256,1,T71,1,749,3,222,381,NaN,0
44788,204,1254,1,T71,0,0,4,NaN,0,481,666


# 取扱貨物量

In [90]:
df_cargo =  df_matster.copy()

# 品名と取扱貨物量に変換
df_cargo['品名'] = df_cargo.apply(lambda x: x['出荷品名'] if x['出荷ﾄﾝ数'] != '0' else x['入荷品名'], 1)
df_cargo['取扱貨物量'] = df_cargo.apply(lambda x: x['出荷ﾄﾝ数'] if x['出荷ﾄﾝ数'] != '0' else x['入荷ﾄﾝ数'], 1)

# 必要な列だけ抽出
columns = ['調査港','施設','品名','取扱貨物量']
df_cargo = df_cargo[columns]

# 取扱貨物量の集計
df_cargo = df_cargo.astype({'取扱貨物量': 'int'})
df_cargo = df_cargo.groupby(['施設','品名']).agg({'取扱貨物量': 'sum'}).reset_index()


# 品名をマージ
df1 = pd.read_csv('品名コード.csv', header=0, encoding="shift-jis",dtype=str)
df_res = pd.merge(df_cargo, df1, on='品名', how='outer')

# 施設名をマージ
df2 = pd.read_csv('施設コード.csv', header=0, encoding="shift-jis",dtype=str)
df_res = pd.merge(df_res, df2, on='施設', how='outer')


# df['cardId'].replace('', np.nan, inplace=True)
df_res.dropna(subset=['取扱貨物量'], inplace=True)


# CSVに書き出し
df_res.to_csv('取扱貨物量.csv',encoding='cp932')

df_res

,施設,品名,取扱貨物量,中分類,大分類,施設名,地区名,港名
0,1153,161,406713.0,砂利・砂,(3)鉱産品,別府公共物揚場\n（砂揚場）(-4.0m),別府港区,東播磨港
1,1153,162,31300.0,石材,(3)鉱産品,別府公共物揚場\n（砂揚場）(-4.0m),別府港区,東播磨港
2,1156,161,375975.0,砂利・砂,(3)鉱産品,高砂公共物揚場(-3.5m),高砂本港区,東播磨港
3,1156,81,19.0,水産品,(1)農水産品,高砂公共物揚場(-3.5m),高砂本港区,東播磨港
4,1157,161,77250.0,砂利・砂,(3)鉱産品,高砂公共岸壁(-5.5m),高砂本港区,東播磨港
...,...,...,...,...,...,...,...,...
135,1222,252,12770.0,完成自動車,(4)金属機械工業品,飾磨１〜２号物揚場(-3.5m),飾磨港区,姫路港
136,1256,222,269325.0,鋼材,(4)金属機械工業品,吉美３号岸壁(-7.5m),網干港区,姫路港
137,1256,423,1262.0,水,(6)軽工業品,吉美３号岸壁(-7.5m),網干港区,姫路港
138,1259,301,2206.0,窯業品,(5)化学工業品,網干港区その他公共施設,網干港区,姫路港


# 係留船舶

In [128]:
import numpy as np

df_ship =  df_matster.copy()

# 必要な列だけ抽出
columns = ['調査港','施設','用途','船舶隻数','船舶総ﾄﾝ数','係留時間']
df_ship = df_ship[columns]

# 欠損値があるので、全てに'1'をセット
df_ship['船舶隻数'] = '1'

# 船舶総ﾄﾝ数が0の行を削除
df_ship['船舶総ﾄﾝ数'].replace('0', np.nan, inplace=True)
df_ship.dropna(subset=['船舶総ﾄﾝ数'], inplace=True)

# 船舶総ﾄﾝ数・係留時間は平均、船舶隻数は合計
df_ship = df_ship.astype({'船舶総ﾄﾝ数': 'int','係留時間': 'int','船舶隻数': 'int'})
df_ship = df_ship.groupby(['施設','用途']).agg({'船舶総ﾄﾝ数': 'mean','係留時間': 'mean','船舶隻数': 'sum'}).reset_index()

# 用途のtをTに変換
df_ship = df_ship.replace('t71', 'T71')
df_ship = df_ship.replace('t85', 'T85')

# 船種をマージ
df1 = pd.read_csv('用途コード.csv', header=0, encoding="shift-jis",dtype=str)
df_res = pd.merge(df_ship, df1, on='用途', how='left')

# 施設名をマージ
df2 = pd.read_csv('施設コード.csv', header=0, encoding="shift-jis",dtype=str)
df_res = pd.merge(df_res, df2, on='施設', how='left')

# CSVに書き出し
df_res.to_csv('係留船舶.csv',encoding='cp932')

df_res

,施設,用途,船舶総ﾄﾝ数,係留時間,船舶隻数,船種,施設名,地区名,港名
0,1153,T71,798.000000,4.000000,1,一般貨物船,別府公共物揚場\n（砂揚場）(-4.0m),別府港区,東播磨港
1,1156,T84,688.778846,6.144231,104,砂利・砂・石材船,高砂公共物揚場(-3.5m),高砂本港区,東播磨港
2,1157,T85,717.701031,4.845361,97,セメント船,高砂公共岸壁(-5.5m),高砂本港区,東播磨港
3,1158,T97,502.000000,14.088889,45,曳船・押船,高砂西港公共岸壁(-5.5m),高砂西港区,東播磨港
4,1160,T84,731.000000,5.000000,1,砂利・砂・石材船,伊保公共物揚場(-3.5m),伊保港区,東播磨港
5,1182,T84,686.805556,4.222222,36,砂利・砂・石材船,曽根公共物揚場(-4.0m),曽根港区,東播磨港
6,1182,T97,2475.000000,5.000000,2,曳船・押船,曽根公共物揚場(-4.0m),曽根港区,東播磨港
7,1212,T71,508.500000,8.500000,2,一般貨物船,中島公共岸壁(-5.5m),飾磨港区,姫路港
8,1212,T85,748.333333,4.000000,3,セメント船,中島公共岸壁(-5.5m),飾磨港区,姫路港
9,1212,T85,748.628571,10.400000,35,セメント船,中島公共岸壁(-5.5m),飾磨港区,姫路港
